In [1]:
import requests
from datetime import datetime
import psycopg2

In [3]:
API_KEY = "7215bf43b5de4fd2a9161700c44d5ee9"
BASE_URL = "https://api.football-data.org/v4/competitions/PL/scorers"
HEADERS = {"X-Auth-Token": API_KEY}

# request scorers
response = requests.get(BASE_URL, headers=HEADERS)
data = response.json()

# Step 2 — connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="bolanlelove",
    dbname="epl"
)
cur = conn.cursor()

# insert/update scorers
for scorer in data['scorers']:
    player = scorer['player']
    team = scorer['team']
    goals = scorer.get('goals', 0)
    assists = scorer.get('assists', 0)

    cur.execute("""
        INSERT INTO scorers (player_id, player_name, team_name, goals, assists, last_update)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (player_id) DO UPDATE SET
            goals = EXCLUDED.goals,
            assists = EXCLUDED.assists,
            last_update = EXCLUDED.last_update;
    """, (
        player['id'],
        player['name'],
        team['name'],
        goals,
        assists,
        datetime.now()
    ))

conn.commit()
